In [1]:
from config.load_config import ConfigGetter

In [2]:
conf = ConfigGetter("lab_mac")
source_dir = conf.data_path
model_dir = conf.model_path

In [3]:
import os
import pandas as pd
import numpy as np
from plotly.offline import iplot, init_notebook_mode, plot
import plotly.graph_objs as go

In [4]:
import keras
from keras.models import model_from_json

from nilearn import image as nilimg
import sys

/Users/yoshilab/.pyenv/versions/3.5.1/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



Using TensorFlow backend.


In [5]:
json_string = open(os.path.join(model_dir, 'model.json')).read()
model = model_from_json(json_string)
model.load_weights(os.path.join(model_dir, 'weights.hdf5'))

In [6]:
SLICE_NUM = 185
MAX_VALUE = 1914.090820

In [7]:
month_list = os.listdir(source_dir)

In [13]:
pat_list_dir = '/Users/yoshilab/PycharmProjects/MRA/structure_data'

In [10]:
predict_list = np.empty((0, 2)).astype(np.float32)

In [14]:
for month in month_list:
    patlist = [patient for patient in os.listdir(os.path.join(source_dir, month))]
    pat_id_list = pd.read_csv(os.path.join(pat_list_dir, month,'patient_data.csv'),
                              header=None,
                              index_col=None, names=['dir', 'pid', 'page', 'psex'],
                              dtype={'pid': 'object'})
    for patient in patlist:
        nifti_files = os.listdir(os.path.join(source_dir, month, patient))
        raw_file = [file for file in nifti_files if file.startswith('2018') & file.endswith('.nii')][0]
        personal_nifti = nilimg.smooth_img(os.path.join(source_dir, month, patient, raw_file), fwhm='fast')
        personal_img = personal_nifti.get_data()
        reshape_img = np.reshape(personal_img, (1, personal_img.shape[0], personal_img.shape[1], personal_img.shape[2]))
        
        if reshape_img.shape[3] < SLICE_NUM:
            train_img = np.append(reshape_img, np.zeros((1, reshape_img.shape[1], reshape_img.shape[2], SLICE_NUM - reshape_img.shape[3])), axis=3)
        elif reshape_img.shape[3] == SLICE_NUM:
            train_img = reshape_img
        else:
            sys.stderr.write('Data Size Error: This Nifti file size is ' + str(reshape_img.shape))
            sys.exit()
        
        personal_age = pat_id_list[pat_id_list.dir == str(patient)]['page'].values[0]
        train_label = [int(personal_age.rstrip('Y'))]
        
        train_img /= MAX_VALUE
        predict_age = model.predict(train_img)
        
        data = np.array([[train_label[0], predict_age[0][0]]])
        predict_list = np.append(predict_list, data, axis=0)

In [16]:
trace = go.Scatter(
    x=predict_list[:, 0],
    y=predict_list[:, 1],
    mode='markers',
    marker=dict(
        color=("#ff4560"),
        size=10
    )
)

data = [trace]

In [21]:
layout = go.Layout(font=dict(size=24),
                   xaxis=dict(title="Real Age",
                              titlefont=dict(size=24),
                              # range=[0, 100]
                              ),
                   yaxis=dict(title="Predict Age",
                              titlefont=dict(size=24),
                              # range=[0, 100]
                              )
                   )

fig = go.Figure(data=data, layout=layout)

In [22]:
plot(fig, filename="html/predict_result_multipleRegression_real.html", image_height=750, image_width=1200, image="svg", auto_open=False)

'file:///Users/yoshilab/PycharmProjects/data_plot/html/predict_result_multipleRegression_real.html'

In [19]:
predict_list

array([[  57.        ,  851.87683105],
       [  43.        ,  616.69763184],
       [  38.        ,  583.62017822],
       [  42.        ,  692.18280029],
       [  46.        ,  739.61682129],
       [  52.        ,  629.10601807],
       [  47.        ,  880.30871582],
       [  63.        ,  613.49645996],
       [  60.        ,  501.00012207],
       [  68.        ,  628.49981689],
       [  31.        ,  659.09771729],
       [  63.        ,  637.53210449],
       [  50.        , 1168.7791748 ],
       [  51.        ,  650.05181885],
       [  55.        ,  664.68572998],
       [  71.        ,  910.62084961],
       [  63.        ,  972.62512207],
       [  47.        ,  568.34454346],
       [  45.        ,  675.9553833 ],
       [  67.        , 1004.46673584],
       [  60.        ,  560.59338379],
       [  27.        ,  526.91131592],
       [  57.        ,  651.5178833 ],
       [  68.        ,  703.91369629],
       [  54.        ,  762.11779785],
       [  79.        ,  7